In [64]:
import re
import pandas as pd
import numpy as np

# from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
#  Sequential,
from tensorflow.keras.layers import Input, Dense, LSTM, Conv1D, AveragePooling1D, MaxPooling1D, Embedding, LSTM, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
#  GRU, Bidirectional
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# from transformers import BertTokenizerFast, TFBertModel
from sentence_transformers import SentenceTransformer
# from transformers import DistilBertTokenizer, TFDistilBertModel

# from scripts.word_embeddings import load_embedding_weights

# Utility functions

In [5]:
def clean(word):
    return re.sub(r'[\-\(\)\?\*\\\'\"\.%\^,:<>|;!]', '', word.lower())

# Preprocessing

In [6]:
path = '/home/petar/Fakultet/Semester 7/NLP/Datasets/fake_news/'

In [7]:
train = pd.read_csv(path + 'full_train_df.csv', index_col=0)
test = pd.read_csv(path + 'full_test_df.csv', index_col=0)

In [8]:
train['tweet'] = train['tweet'].apply(lambda x: clean(x))
test['tweet'] = test['tweet'].apply(lambda x: clean(x))

In [9]:
train = train.groupby(['user', 'label'])['tweet'].agg(list).reset_index()
test = test.groupby(['user', 'label'])['tweet'].agg(list).reset_index()

In [10]:
all_sentences = pd.Series.append(train['tweet'].copy(), test['tweet'].copy())

/tmp/ipykernel_19434/4018911843.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_sentences = pd.Series.append(train['tweet'].copy(), test['tweet'].copy())


In [14]:
sentences = all_sentences.reset_index()['tweet']

In [12]:
sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

In [15]:
embeddings = sentence_transformer.encode(sentences)

In [17]:
embeddings.shape

(500, 384)

In [18]:
train['embedding'] = np.copy(embeddings[:300, :]).tolist()
test['embedding'] = np.copy(embeddings[300:, :]).tolist()

In [22]:
x_train = range(len(train))
y_train = train['label'].copy().values

x_test = range(len(test))
y_test = test['label'].copy().values

In [23]:
x = np.append(x_train, x_test)
y = np.append(y_train, y_test)

In [26]:
x = range(len(x))

In [33]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.4, shuffle=True, stratify=y, random_state=0)

In [113]:
# x_train = np.asarray(x_train.apply(lambda emb: np.array(emb)))
# y_train = y_train.to_numpy().reshape(-1, 1)
# x_train = x_train.tolist()
# y_train = y_train.tolist()

# Models

In [40]:
model1 = Sequential()

model1.add(Embedding(input_dim=len(embeddings), output_dim=embeddings.shape[1], input_length=1, weights=[embeddings], trainable=True))

model1.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))
model1.add(GlobalAveragePooling1D())


model1.add(Dense(64, activation='relu'))
model1.add(Dense(32, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))


model1.compile(optimizer=Adam(learning_rate=.00001), loss='binary_crossentropy', metrics=['accuracy'])

model1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1, 384)            192000    
                                                                 
 conv1d_4 (Conv1D)           (None, 1, 128)            1572992   
                                                                 
 global_average_pooling1d_3   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                      

In [41]:
model1.fit(x_train, y_train.tolist(), epochs=50, batch_size=32, verbose=2)

Epoch 1/50
10/10 - 2s - loss: 0.6932 - accuracy: 0.5133 - 2s/epoch - 152ms/step
Epoch 2/50
10/10 - 1s - loss: 0.6930 - accuracy: 0.5400 - 966ms/epoch - 97ms/step
Epoch 3/50
10/10 - 1s - loss: 0.6929 - accuracy: 0.5533 - 893ms/epoch - 89ms/step
Epoch 4/50
10/10 - 1s - loss: 0.6927 - accuracy: 0.5800 - 986ms/epoch - 99ms/step
Epoch 5/50
10/10 - 1s - loss: 0.6926 - accuracy: 0.5967 - 878ms/epoch - 88ms/step
Epoch 6/50
10/10 - 1s - loss: 0.6925 - accuracy: 0.6067 - 876ms/epoch - 88ms/step
Epoch 7/50
10/10 - 1s - loss: 0.6924 - accuracy: 0.6133 - 927ms/epoch - 93ms/step
Epoch 8/50
10/10 - 1s - loss: 0.6923 - accuracy: 0.6300 - 1s/epoch - 100ms/step
Epoch 9/50
10/10 - 1s - loss: 0.6921 - accuracy: 0.6433 - 870ms/epoch - 87ms/step
Epoch 10/50
10/10 - 1s - loss: 0.6920 - accuracy: 0.6600 - 872ms/epoch - 87ms/step
Epoch 11/50
10/10 - 1s - loss: 0.6919 - accuracy: 0.6833 - 874ms/epoch - 87ms/step
Epoch 12/50
10/10 - 1s - loss: 0.6917 - accuracy: 0.7067 - 952ms/epoch - 95ms/step
Epoch 13/50
10/10

In [74]:
y_pred = model1.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0       0.60      0.54      0.57       100
           1       0.58      0.64      0.61       100

    accuracy                           0.59       200
   macro avg       0.59      0.59      0.59       200
weighted avg       0.59      0.59      0.59       200



# Model2

In [76]:
model2 = Sequential()

model2.add(Embedding(input_dim=len(embeddings), output_dim=embeddings.shape[1], input_length=1, weights=[embeddings], trainable=True))

model2.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))

model2.add(LSTM(128))

model2.add(Dense(64, activation='relu'))
model2.add(Dense(32, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))


model2.compile(optimizer=Adam(learning_rate=.00001), loss='binary_crossentropy', metrics=['accuracy'])

model2.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 1, 384)            192000    
                                                                 
 conv1d_29 (Conv1D)          (None, 1, 128)            1572992   
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dense_15 (Dense)            (None, 64)                8256      
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,906,945
Trainable params: 1,906,945
N

In [77]:
model2.fit(x_train, y_train.tolist(), epochs=50, batch_size=32, verbose=2)

Epoch 1/50
10/10 - 3s - loss: 0.6932 - accuracy: 0.4500 - 3s/epoch - 265ms/step
Epoch 2/50
10/10 - 1s - loss: 0.6931 - accuracy: 0.4867 - 1s/epoch - 113ms/step
Epoch 3/50
10/10 - 1s - loss: 0.6931 - accuracy: 0.5233 - 949ms/epoch - 95ms/step
Epoch 4/50
10/10 - 1s - loss: 0.6931 - accuracy: 0.5400 - 956ms/epoch - 96ms/step
Epoch 5/50
10/10 - 1s - loss: 0.6930 - accuracy: 0.5600 - 1s/epoch - 117ms/step
Epoch 6/50
10/10 - 1s - loss: 0.6930 - accuracy: 0.6067 - 1s/epoch - 105ms/step
Epoch 7/50
10/10 - 1s - loss: 0.6930 - accuracy: 0.6167 - 967ms/epoch - 97ms/step
Epoch 8/50
10/10 - 1s - loss: 0.6929 - accuracy: 0.6333 - 959ms/epoch - 96ms/step
Epoch 9/50
10/10 - 1s - loss: 0.6929 - accuracy: 0.6400 - 920ms/epoch - 92ms/step
Epoch 10/50
10/10 - 1s - loss: 0.6929 - accuracy: 0.6500 - 1s/epoch - 100ms/step
Epoch 11/50
10/10 - 1s - loss: 0.6928 - accuracy: 0.6700 - 952ms/epoch - 95ms/step
Epoch 12/50
10/10 - 1s - loss: 0.6928 - accuracy: 0.6867 - 929ms/epoch - 93ms/step
Epoch 13/50
10/10 - 1s 

In [78]:
y_pred = model2.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0       0.64      0.50      0.56       100
           1       0.59      0.72      0.65       100

    accuracy                           0.61       200
   macro avg       0.62      0.61      0.61       200
weighted avg       0.62      0.61      0.61       200



# Model3

In [80]:
model3 = Sequential()

model3.add(Embedding(input_dim=len(embeddings), output_dim=embeddings.shape[1], input_length=1, weights=[embeddings], trainable=True))

model3.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))
model3.add(Conv1D(64, 16, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))

model3.add(LSTM(128))

model3.add(Dense(64, activation='relu'))
model3.add(Dense(32, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))


model3.compile(optimizer=Adam(learning_rate=.00001), loss='binary_crossentropy', metrics=['accuracy'])

model3.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, 1, 384)            192000    
                                                                 
 conv1d_32 (Conv1D)          (None, 1, 128)            1572992   
                                                                 
 conv1d_33 (Conv1D)          (None, 1, 64)             131136    
                                                                 
 lstm_5 (LSTM)               (None, 128)               98816     
                                                                 
 dense_21 (Dense)            (None, 64)                8256      
                                                                 
 dense_22 (Dense)            (None, 32)                2080      
                                                                 
 dense_23 (Dense)            (None, 1)               

In [81]:
model3.fit(x_train, y_train.tolist(), epochs=50, batch_size=32, verbose=2)

Epoch 1/50
10/10 - 3s - loss: 0.6931 - accuracy: 0.4933 - 3s/epoch - 279ms/step
Epoch 2/50
10/10 - 1s - loss: 0.6931 - accuracy: 0.5333 - 1s/epoch - 101ms/step
Epoch 3/50
10/10 - 1s - loss: 0.6931 - accuracy: 0.5833 - 902ms/epoch - 90ms/step
Epoch 4/50
10/10 - 1s - loss: 0.6931 - accuracy: 0.5933 - 929ms/epoch - 93ms/step
Epoch 5/50
10/10 - 1s - loss: 0.6931 - accuracy: 0.5967 - 884ms/epoch - 88ms/step
Epoch 6/50
10/10 - 1s - loss: 0.6931 - accuracy: 0.6233 - 1s/epoch - 101ms/step
Epoch 7/50
10/10 - 1s - loss: 0.6931 - accuracy: 0.5633 - 997ms/epoch - 100ms/step
Epoch 8/50
10/10 - 1s - loss: 0.6931 - accuracy: 0.5300 - 914ms/epoch - 91ms/step
Epoch 9/50
10/10 - 1s - loss: 0.6931 - accuracy: 0.5900 - 986ms/epoch - 99ms/step
Epoch 10/50
10/10 - 1s - loss: 0.6931 - accuracy: 0.6233 - 1s/epoch - 104ms/step
Epoch 11/50
10/10 - 1s - loss: 0.6931 - accuracy: 0.6100 - 938ms/epoch - 94ms/step
Epoch 12/50
10/10 - 1s - loss: 0.6930 - accuracy: 0.6467 - 1s/epoch - 107ms/step
Epoch 13/50
10/10 - 1s

In [82]:
y_pred = model3.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

           0       0.58      0.63      0.61       100
           1       0.60      0.55      0.57       100

    accuracy                           0.59       200
   macro avg       0.59      0.59      0.59       200
weighted avg       0.59      0.59      0.59       200



# Adding a CNN layer doesn't improve classification

# Model4

# Learning rate is changed from .00001 -> .0001

In [83]:
model4 = Sequential()

model4.add(Embedding(input_dim=len(embeddings), output_dim=embeddings.shape[1], input_length=1, weights=[embeddings], trainable=True))

model4.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))

model4.add(LSTM(128, return_sequences=True))
model4.add(LSTM(128))

model4.add(Dense(64, activation='relu'))
model4.add(Dense(32, activation='relu'))
model4.add(Dense(1, activation='sigmoid'))


model4.compile(optimizer=Adam(learning_rate=.0001), loss='binary_crossentropy', metrics=['accuracy'])

model4.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_25 (Embedding)    (None, 1, 384)            192000    
                                                                 
 conv1d_34 (Conv1D)          (None, 1, 128)            1572992   
                                                                 
 lstm_6 (LSTM)               (None, 1, 128)            131584    
                                                                 
 lstm_7 (LSTM)               (None, 128)               131584    
                                                                 
 dense_24 (Dense)            (None, 64)                8256      
                                                                 
 dense_25 (Dense)            (None, 32)                2080      
                                                                 
 dense_26 (Dense)            (None, 1)               

In [84]:
model4.fit(x_train, y_train.tolist(), epochs=50, batch_size=32, verbose=2)

Epoch 1/50
10/10 - 4s - loss: 0.6932 - accuracy: 0.5000 - 4s/epoch - 420ms/step
Epoch 2/50
10/10 - 1s - loss: 0.6931 - accuracy: 0.5000 - 1s/epoch - 108ms/step
Epoch 3/50
10/10 - 1s - loss: 0.6930 - accuracy: 0.5000 - 1s/epoch - 111ms/step
Epoch 4/50
10/10 - 1s - loss: 0.6929 - accuracy: 0.5000 - 1s/epoch - 105ms/step
Epoch 5/50
10/10 - 1s - loss: 0.6928 - accuracy: 0.6600 - 918ms/epoch - 92ms/step
Epoch 6/50
10/10 - 1s - loss: 0.6925 - accuracy: 0.5900 - 918ms/epoch - 92ms/step
Epoch 7/50
10/10 - 1s - loss: 0.6919 - accuracy: 0.7533 - 911ms/epoch - 91ms/step
Epoch 8/50
10/10 - 1s - loss: 0.6910 - accuracy: 0.7700 - 908ms/epoch - 91ms/step
Epoch 9/50
10/10 - 1s - loss: 0.6894 - accuracy: 0.8000 - 979ms/epoch - 98ms/step
Epoch 10/50
10/10 - 1s - loss: 0.6869 - accuracy: 0.8067 - 1s/epoch - 108ms/step
Epoch 11/50
10/10 - 1s - loss: 0.6827 - accuracy: 0.8667 - 1s/epoch - 104ms/step
Epoch 12/50
10/10 - 1s - loss: 0.6759 - accuracy: 0.8700 - 927ms/epoch - 93ms/step
Epoch 13/50
10/10 - 1s - 

In [85]:
y_pred = model4.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

           0       0.59      0.87      0.70       100
           1       0.75      0.39      0.51       100

    accuracy                           0.63       200
   macro avg       0.67      0.63      0.61       200
weighted avg       0.67      0.63      0.61       200



# Better results, but needs less training.

# Model5

# The same model as model4, but less epochs to prevent overfitting.

In [86]:
model5 = Sequential()

model5.add(Embedding(input_dim=len(embeddings), output_dim=embeddings.shape[1], input_length=1, weights=[embeddings], trainable=True))

model5.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))


model5.add(LSTM(128, return_sequences=True))
model5.add(LSTM(128))

model5.add(Dense(64, activation='relu'))
model5.add(Dense(32, activation='relu'))
model5.add(Dense(1, activation='sigmoid'))


model5.compile(optimizer=Adam(learning_rate=.0001), loss='binary_crossentropy', metrics=['accuracy'])

model5.summary()

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_26 (Embedding)    (None, 1, 384)            192000    
                                                                 
 conv1d_35 (Conv1D)          (None, 1, 128)            1572992   
                                                                 
 lstm_8 (LSTM)               (None, 1, 128)            131584    
                                                                 
 lstm_9 (LSTM)               (None, 128)               131584    
                                                                 
 dense_27 (Dense)            (None, 64)                8256      
                                                                 
 dense_28 (Dense)            (None, 32)                2080      
                                                                 
 dense_29 (Dense)            (None, 1)               

In [88]:
model5.fit(x_train, y_train.tolist(), epochs=20, batch_size=32, verbose=2)

Epoch 1/20
10/10 - 4s - loss: 0.6932 - accuracy: 0.4833 - 4s/epoch - 378ms/step
Epoch 2/20
10/10 - 1s - loss: 0.6931 - accuracy: 0.5000 - 993ms/epoch - 99ms/step
Epoch 3/20
10/10 - 1s - loss: 0.6931 - accuracy: 0.5000 - 909ms/epoch - 91ms/step
Epoch 4/20
10/10 - 1s - loss: 0.6930 - accuracy: 0.5033 - 916ms/epoch - 92ms/step
Epoch 5/20
10/10 - 1s - loss: 0.6928 - accuracy: 0.6267 - 908ms/epoch - 91ms/step
Epoch 6/20
10/10 - 1s - loss: 0.6926 - accuracy: 0.7067 - 907ms/epoch - 91ms/step
Epoch 7/20
10/10 - 1s - loss: 0.6921 - accuracy: 0.6700 - 911ms/epoch - 91ms/step
Epoch 8/20
10/10 - 1s - loss: 0.6911 - accuracy: 0.7167 - 923ms/epoch - 92ms/step
Epoch 9/20
10/10 - 1s - loss: 0.6896 - accuracy: 0.7400 - 905ms/epoch - 91ms/step
Epoch 10/20
10/10 - 1s - loss: 0.6871 - accuracy: 0.7800 - 893ms/epoch - 89ms/step
Epoch 11/20
10/10 - 1s - loss: 0.6829 - accuracy: 0.7933 - 911ms/epoch - 91ms/step
Epoch 12/20
10/10 - 1s - loss: 0.6758 - accuracy: 0.8300 - 905ms/epoch - 91ms/step
Epoch 13/20
10/

In [89]:
y_pred = model5.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

           0       0.56      0.88      0.68       100
           1       0.71      0.30      0.42       100

    accuracy                           0.59       200
   macro avg       0.64      0.59      0.55       200
weighted avg       0.64      0.59      0.55       200



# Less training didn't improve results.

# Model6

# Adding more dense layers

In [92]:
model6 = Sequential()

model6.add(Embedding(input_dim=len(embeddings), output_dim=embeddings.shape[1], input_length=1, weights=[embeddings], trainable=True))

model6.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))


model6.add(LSTM(128, return_sequences=True))
model6.add(LSTM(128))

model6.add(Dense(64, activation='relu'))
model6.add(Dense(32, activation='relu'))
model6.add(Dense(32, activation='relu'))
model6.add(Dense(32, activation='relu'))
model6.add(Dense(1, activation='sigmoid'))


model6.compile(optimizer=Adam(learning_rate=.0001), loss='binary_crossentropy', metrics=['accuracy'])

model6.summary()

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_28 (Embedding)    (None, 1, 384)            192000    
                                                                 
 conv1d_37 (Conv1D)          (None, 1, 128)            1572992   
                                                                 
 lstm_12 (LSTM)              (None, 1, 128)            131584    
                                                                 
 lstm_13 (LSTM)              (None, 128)               131584    
                                                                 
 dense_35 (Dense)            (None, 64)                8256      
                                                                 
 dense_36 (Dense)            (None, 32)                2080      
                                                                 
 dense_37 (Dense)            (None, 32)              

In [93]:
model6.fit(x_train, y_train.tolist(), epochs=30, batch_size=32, verbose=2)

Epoch 1/30
10/10 - 4s - loss: 0.6932 - accuracy: 0.4667 - 4s/epoch - 397ms/step
Epoch 2/30
10/10 - 1s - loss: 0.6931 - accuracy: 0.5000 - 1s/epoch - 112ms/step
Epoch 3/30
10/10 - 1s - loss: 0.6931 - accuracy: 0.5067 - 957ms/epoch - 96ms/step
Epoch 4/30
10/10 - 1s - loss: 0.6931 - accuracy: 0.5167 - 1s/epoch - 108ms/step
Epoch 5/30
10/10 - 1s - loss: 0.6931 - accuracy: 0.5000 - 1s/epoch - 108ms/step
Epoch 6/30
10/10 - 1s - loss: 0.6931 - accuracy: 0.5000 - 918ms/epoch - 92ms/step
Epoch 7/30
10/10 - 1s - loss: 0.6930 - accuracy: 0.5000 - 931ms/epoch - 93ms/step
Epoch 8/30
10/10 - 1s - loss: 0.6930 - accuracy: 0.6000 - 1s/epoch - 105ms/step
Epoch 9/30
10/10 - 1s - loss: 0.6928 - accuracy: 0.6433 - 944ms/epoch - 94ms/step
Epoch 10/30
10/10 - 1s - loss: 0.6924 - accuracy: 0.8033 - 922ms/epoch - 92ms/step
Epoch 11/30
10/10 - 1s - loss: 0.6915 - accuracy: 0.8300 - 1s/epoch - 104ms/step
Epoch 12/30
10/10 - 1s - loss: 0.6900 - accuracy: 0.8767 - 985ms/epoch - 98ms/step
Epoch 13/30
10/10 - 1s - 

In [95]:
y_pred = model6.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

           0       0.59      0.61      0.60       100
           1       0.59      0.57      0.58       100

    accuracy                           0.59       200
   macro avg       0.59      0.59      0.59       200
weighted avg       0.59      0.59      0.59       200



# Results more or less still the same.

# Model7

# Learning rate is changed from .0001 -> .001 . Epochs 30 -> 20.

In [100]:
model7 = Sequential()

model7.add(Embedding(input_dim=len(embeddings), output_dim=embeddings.shape[1], input_length=1, weights=[embeddings], trainable=True))

model7.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))


model7.add(LSTM(128, return_sequences=True))
model7.add(LSTM(128))

model7.add(Dense(64, activation='relu'))
model7.add(Dense(32, activation='relu'))
model7.add(Dense(1, activation='sigmoid'))


model7.compile(optimizer=Adam(learning_rate=.001), loss='binary_crossentropy', metrics=['accuracy'])

model7.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_31 (Embedding)    (None, 1, 384)            192000    
                                                                 
 conv1d_40 (Conv1D)          (None, 1, 128)            1572992   
                                                                 
 lstm_18 (LSTM)              (None, 1, 128)            131584    
                                                                 
 lstm_19 (LSTM)              (None, 128)               131584    
                                                                 
 dense_46 (Dense)            (None, 64)                8256      
                                                                 
 dense_47 (Dense)            (None, 32)                2080      
                                                                 
 dense_48 (Dense)            (None, 1)               

In [101]:
model7.fit(x_train, y_train.tolist(), epochs=20, batch_size=32, verbose=2)

Epoch 1/20
10/10 - 4s - loss: 0.6933 - accuracy: 0.4633 - 4s/epoch - 392ms/step
Epoch 2/20
10/10 - 1s - loss: 0.6921 - accuracy: 0.7367 - 1s/epoch - 102ms/step
Epoch 3/20
10/10 - 1s - loss: 0.6756 - accuracy: 0.8467 - 899ms/epoch - 90ms/step
Epoch 4/20
10/10 - 1s - loss: 0.5370 - accuracy: 0.9433 - 886ms/epoch - 89ms/step
Epoch 5/20
10/10 - 1s - loss: 0.1492 - accuracy: 0.9867 - 895ms/epoch - 90ms/step
Epoch 6/20
10/10 - 1s - loss: 0.0021 - accuracy: 1.0000 - 1s/epoch - 104ms/step
Epoch 7/20
10/10 - 1s - loss: 3.1276e-05 - accuracy: 1.0000 - 899ms/epoch - 90ms/step
Epoch 8/20
10/10 - 1s - loss: 3.6642e-06 - accuracy: 1.0000 - 892ms/epoch - 89ms/step
Epoch 9/20
10/10 - 1s - loss: 1.7805e-06 - accuracy: 1.0000 - 891ms/epoch - 89ms/step
Epoch 10/20
10/10 - 1s - loss: 1.3077e-06 - accuracy: 1.0000 - 889ms/epoch - 89ms/step
Epoch 11/20
10/10 - 1s - loss: 1.1732e-06 - accuracy: 1.0000 - 898ms/epoch - 90ms/step
Epoch 12/20
10/10 - 1s - loss: 1.1135e-06 - accuracy: 1.0000 - 907ms/epoch - 91ms/

In [102]:
y_pred = model7.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

           0       0.57      0.91      0.70       100
           1       0.78      0.32      0.45       100

    accuracy                           0.61       200
   macro avg       0.68      0.61      0.58       200
weighted avg       0.68      0.61      0.58       200



# Results are better but are not satisfactory.

# Model8

# Again, we will add more CNN layers

In [107]:
model8 = Sequential()

model8.add(Embedding(input_dim=len(embeddings), output_dim=embeddings.shape[1], input_length=1, weights=[embeddings], trainable=True))

model8.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))
model8.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))
model8.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))


model8.add(LSTM(128, return_sequences=True))
model8.add(LSTM(128))

model8.add(Dense(64, activation='relu'))
model8.add(Dense(32, activation='relu'))
model8.add(Dense(1, activation='sigmoid'))


model8.compile(optimizer=Adam(learning_rate=.001), loss='binary_crossentropy', metrics=['accuracy'])

model8.summary()

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_33 (Embedding)    (None, 1, 384)            192000    
                                                                 
 conv1d_44 (Conv1D)          (None, 1, 128)            1572992   
                                                                 
 conv1d_45 (Conv1D)          (None, 1, 128)            524416    
                                                                 
 conv1d_46 (Conv1D)          (None, 1, 128)            524416    
                                                                 
 lstm_22 (LSTM)              (None, 1, 128)            131584    
                                                                 
 lstm_23 (LSTM)              (None, 128)               131584    
                                                                 
 dense_52 (Dense)            (None, 64)              

In [108]:
model8.fit(x_train, y_train.tolist(), epochs=20, batch_size=32, verbose=2)

Epoch 1/20
10/10 - 22s - loss: 0.6934 - accuracy: 0.5000 - 22s/epoch - 2s/step
Epoch 2/20
10/10 - 1s - loss: 0.6932 - accuracy: 0.5000 - 1s/epoch - 142ms/step
Epoch 3/20
10/10 - 1s - loss: 0.6934 - accuracy: 0.4800 - 1s/epoch - 141ms/step
Epoch 4/20
10/10 - 1s - loss: 0.6932 - accuracy: 0.5000 - 1s/epoch - 143ms/step
Epoch 5/20
10/10 - 1s - loss: 0.6932 - accuracy: 0.5000 - 1s/epoch - 142ms/step
Epoch 6/20
10/10 - 1s - loss: 0.6932 - accuracy: 0.5000 - 1s/epoch - 142ms/step
Epoch 7/20
10/10 - 1s - loss: 0.6932 - accuracy: 0.5333 - 1s/epoch - 144ms/step
Epoch 8/20
10/10 - 1s - loss: 0.6803 - accuracy: 0.7933 - 1s/epoch - 144ms/step
Epoch 9/20
10/10 - 1s - loss: 0.4031 - accuracy: 0.9533 - 1s/epoch - 144ms/step
Epoch 10/20
10/10 - 1s - loss: 0.0757 - accuracy: 0.9900 - 1s/epoch - 143ms/step
Epoch 11/20
10/10 - 1s - loss: 3.7930e-04 - accuracy: 1.0000 - 1s/epoch - 139ms/step
Epoch 12/20
10/10 - 1s - loss: 2.4813e-06 - accuracy: 1.0000 - 1s/epoch - 142ms/step
Epoch 13/20
10/10 - 2s - loss:

In [109]:
y_pred = model8.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 1s 5ms/step
              precision    recall  f1-score   support

           0       0.59      0.64      0.62       100
           1       0.61      0.56      0.58       100

    accuracy                           0.60       200
   macro avg       0.60      0.60      0.60       200
weighted avg       0.60      0.60      0.60       200



# Again, no improvements with more CNN, despite higher learning rate.

# Model9

# The same model as before, but smaller batch_size 32 -> 16

In [110]:
model9 = Sequential()

model9.add(Embedding(input_dim=len(embeddings), output_dim=embeddings.shape[1], input_length=1, weights=[embeddings], trainable=True))

model9.add(Conv1D(128, 32, padding='same', activation=tf.keras.layers.LeakyReLU(alpha=.1)))


model9.add(LSTM(128, return_sequences=True))
model9.add(LSTM(128))

model9.add(Dense(64, activation='relu'))
model9.add(Dense(32, activation='relu'))
model9.add(Dense(1, activation='sigmoid'))


model9.compile(optimizer=Adam(learning_rate=.001), loss='binary_crossentropy', metrics=['accuracy'])

model9.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_34 (Embedding)    (None, 1, 384)            192000    
                                                                 
 conv1d_47 (Conv1D)          (None, 1, 128)            1572992   
                                                                 
 lstm_24 (LSTM)              (None, 1, 128)            131584    
                                                                 
 lstm_25 (LSTM)              (None, 128)               131584    
                                                                 
 dense_55 (Dense)            (None, 64)                8256      
                                                                 
 dense_56 (Dense)            (None, 32)                2080      
                                                                 
 dense_57 (Dense)            (None, 1)               

In [111]:
model9.fit(x_train, y_train.tolist(), epochs=20, batch_size=16, verbose=2)

Epoch 1/20
19/19 - 4s - loss: 0.6932 - accuracy: 0.4667 - 4s/epoch - 232ms/step
Epoch 2/20
19/19 - 1s - loss: 0.6925 - accuracy: 0.6467 - 1s/epoch - 56ms/step
Epoch 3/20
19/19 - 1s - loss: 0.6479 - accuracy: 0.7867 - 1s/epoch - 58ms/step
Epoch 4/20
19/19 - 1s - loss: 0.2869 - accuracy: 0.9533 - 1s/epoch - 55ms/step
Epoch 5/20
19/19 - 1s - loss: 0.0057 - accuracy: 1.0000 - 1s/epoch - 61ms/step
Epoch 6/20
19/19 - 1s - loss: 5.0337e-05 - accuracy: 1.0000 - 1s/epoch - 53ms/step
Epoch 7/20
19/19 - 1s - loss: 2.9441e-05 - accuracy: 1.0000 - 1s/epoch - 55ms/step
Epoch 8/20
19/19 - 1s - loss: 2.2181e-05 - accuracy: 1.0000 - 1s/epoch - 60ms/step
Epoch 9/20
19/19 - 1s - loss: 1.7437e-05 - accuracy: 1.0000 - 1s/epoch - 54ms/step
Epoch 10/20
19/19 - 1s - loss: 1.3864e-05 - accuracy: 1.0000 - 999ms/epoch - 53ms/step
Epoch 11/20
19/19 - 1s - loss: 1.1207e-05 - accuracy: 1.0000 - 1s/epoch - 53ms/step
Epoch 12/20
19/19 - 1s - loss: 8.9649e-06 - accuracy: 1.0000 - 1s/epoch - 57ms/step
Epoch 13/20
19/19

In [112]:
y_pred = model9.predict(x_test)
ypred = y_pred.reshape(len(y_pred), )
y_pred = np.round(y_pred)
print(classification_report(y_test, y_pred))

7/7 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

           0       0.54      0.90      0.68       100
           1       0.71      0.24      0.36       100

    accuracy                           0.57       200
   macro avg       0.62      0.57      0.52       200
weighted avg       0.62      0.57      0.52       200



# More or less the same.